In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [2]:
import numpy as np
import common_plots
import plotly.graph_objects as go
import factors
import factors_sampling
import factors_inference

# Searching for PGM structure
Usually there is uncertainty in problems about which variable is conditionally independent of which other variables, so it might be necessary to search the space of possible graphs to do this. One could use the likelihood of the data under the model to decide which graph is good but this can lead to overfitting. Searching can be done with any number of algorithms. <br>
**Simple Example, Testing a few different Graphs:**

In [155]:
# truth
factorA = factors.Factor(["A"],[2])
factorB = factors.Factor(["B"],[3])
factorC_givenAB = factors.Factor(["C","A","B"],[2,2,3])
factorA.set_all([0.5,0.5])
factorB.set_all([[0.5,0.2,0.3]])
factorC_givenAB.set_all([[0.2,0.1,0.3,0.5,0.4,0.9,0.8,0.9,0.7,0.5,0.6,0.1]])
true_factors = [factorA,factorB,factorC_givenAB]

In [156]:
samples = []
for s in range(1000):
    sample_variable_names,sample = factors_sampling.joint_sample_top_down(true_factors)
    samples.append(sample)
samples = np.array(samples)
samples[np.random.rand(*samples.shape)<0.7]=-1

In [185]:
# Variables e.g ["A","B","C"]
# Parents e.g [[],[],["A","B"]]
# would be A -> C <- B
def make_directed_pgm(variable_names,possible_values,parent_names):
    all_factors = []
    for j in range(len(variable_names)):
        names_j = [name for name in list(variable_names[j])+parent_names[j]]
        pos_values_j = [possible_values[variable_names.index(name)] for name in names_j]
        f = factors.Factor(names_j,pos_values_j)
        all_factors.append(f)
    return all_factors

def randomize_prior(prior_factors):
    for j in range(len(prior_factors)):
        prior_factors[j].array = np.random.rand(*prior_factors[j].array.shape)
        prior_factors[j] = factors.condition(prior_factors[j],prior_factors[j].names[1:])
    return prior_factors

For $A \rightarrow C \leftarrow B$

In [193]:
prior_factors = randomize_prior(make_directed_pgm(["A","B","C"],[2,3,2],[[],[],["A","B"]]))
new_factors = factors_inference.learn_directed_PGM_EM(prior_factors,sample_variable_names,samples,5)

log likelihood -146.05920645506487
log likelihood -109.69558273153234
log likelihood -92.44868181956065
log likelihood -82.92538031797052
log likelihood -76.9864810118957


For $A \rightarrow B \leftarrow C$

In [194]:
prior_factors = randomize_prior(make_directed_pgm(["A","B","C"],[2,3,2],[[],["A","C"],[]]))
new_factors = factors_inference.learn_directed_PGM_EM(prior_factors,sample_variable_names,samples,5)

log likelihood -277.6260437599606
log likelihood -238.47026748374145
log likelihood -220.39124845511552
log likelihood -211.5794463280546
log likelihood -207.11992946319242


For $C \rightarrow A \leftarrow B$

In [195]:
prior_factors = randomize_prior(make_directed_pgm(["A","B","C"],[2,3,2],[["B","C"],[],[]]))
new_factors = factors_inference.learn_directed_PGM_EM(prior_factors,sample_variable_names,samples,5)

log likelihood -300.45722890797396
log likelihood -157.37013235737047
log likelihood -111.10023188695229
log likelihood -91.02092546655243
log likelihood -81.28447237198691


Almost all the time the true graph comes out with the greatest likelihood, though the randomness can affect it.

### Equivalence between PGMs
There are many different ways to expand a probability with the product rule, which means many different graphs encode the same information. For instance, one expansion of $P(A,B,C)$:
$$P(A,B,C)=P(A|B,C)P(B|C)P(C)$$
And another:
$$P(A,B,C)=P(B|A,C)P(C|A)P(A)$$